<a href="https://colab.research.google.com/github/dinhtruongng/VNese-Aspect-based-Sentiment-Analysis/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

In [4]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer 
from skmultilearn.adapt import MLkNN 
from sklearn.metrics import hamming_loss, accuracy_score

import nltk
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import re

In [5]:
train_df = pd.read_csv('Train.csv')
dev_df = pd.read_csv("Dev.csv")
test_df = pd.read_csv("Test.csv")

In [6]:
# Concatenate rows
FULL = pd.concat([train_df, test_df], ignore_index=True)

In [7]:
vetorizar = TfidfVectorizer(max_features=3000, max_df=0.85) 

# Lowercase

In [9]:
def lowercase(df):
    df['comment'] = df['comment'].str.lower()

In [10]:
lowercase(FULL)

In [11]:
# vetorizar = TfidfVectorizer(max_features=3000, max_df=0.85)
# vetorizar.fit(X_train)
# X_train_tfidf = vetorizar.transform(X_train)
# X_test_tfidf = vetorizar.transform(X_test)
# X_train_tfidf

In [12]:
# mlknn_classifier = MLkNN()
# mlknn_classifier.fit(X_train_tfidf, y_train)

# Removing Punctuations

In [14]:
import string
punc = string.punctuation

In [15]:
def remove_punc(text):
    return text.translate(str.maketrans('', '', punc))

In [16]:
def final_rmv_punc(df):
    df['comment'] = df['comment'].apply(remove_punc)

In [17]:
final_rmv_punc(FULL)

# Removing Number?

In [19]:
def remove_num(df):
    df['comment'] = df['comment'].replace(to_replace=r'\d', value='', regex=True)

In [20]:
remove_num(FULL)

In [21]:
train_df.head()

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


# Tokenization

In [23]:
# !pip install pyvi

In [24]:
from pyvi import ViTokenizer, ViPosTagger

In [25]:
tokenizer = ViTokenizer.tokenize

In [26]:
def tokenize(df):
    df['comment'] = df['comment'].apply(tokenizer)

In [27]:
tokenize(FULL)

In [28]:
train_df.head()

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


# POS tagging

In [30]:
tagger = ViPosTagger.postagging

In [31]:
ex = FULL.iloc[0]['comment']

In [32]:
tagger(ex)

(['mới',
  'mua',
  'máy',
  'này',
  'tại',
  'thegioididong',
  'thốt_nốt',
  'cảm_thấy',
  'ok',
  'bin',
  'trâu',
  'chụp',
  'ảnh',
  'đẹp',
  'loa',
  'nghe',
  'to',
  'bắt',
  'wf',
  'khỏe_sóng',
  'ổn_định',
  'giá_thành',
  'vừa',
  'với',
  'túi_tiền',
  'nhân_viên',
  'tư_vấn',
  'nhiệt_tình'],
 array(['R', 'V', 'N', 'P', 'E', 'N', 'N', 'V', 'N', 'V', 'N', 'V', 'N',
        'A', 'N', 'V', 'A', 'V', 'N', 'N', 'A', 'N', 'R', 'E', 'V', 'N',
        'V', 'A'], dtype=object))

In [33]:
def remove_emote(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U00002702-\U000027B0"  # other symbols
        u"\U000024C2-\U0001F251"  # enclosed characters
        u"\U0001f926-\U0001f937"  # gestures
        u"\U0001F1F2"             # specific characters
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"          # gender symbols
        "]+", flags=re.UNICODE)
    
    # Substitute emojis with a space
    text = emoji_pattern.sub(r" ", text)
    return text
def final_remove_emote(df):
    df['comment'] = df['comment'].apply(remove_emote)    
    return df
FULL = final_remove_emote(train_df)


In [34]:
X = FULL['comment']
y = FULL.iloc[:, -1]

In [35]:
label = {'BATTERY':[],'CAMERA':[],'DESIGN':[],'FEATURES':[],'GENERAL':[],'OTHERS':[],'PERFORMANCE':[],'PRICE':[],'SCREEN':[],'SER&ACC':[],'STORAGE':[]}
def spliY(y, label):
    label = {'BATTERY':[],'CAMERA':[],'DESIGN':[],'FEATURES':[],'GENERAL':[],'OTHERS':[],'PERFORMANCE':[],'PRICE':[],'SCREEN':[],'SER&ACC':[],'STORAGE':[]}
    for i in y:
        x = i[:-1].split(";")
        # print(x)
        temp = list(label.keys())
        for j in x:
            j = j.replace('{', '')
            j = j.replace('}', '')
            t = j.split('#')
            
            if t[0] != "OTHERS":
                temp.remove(t[0])
                if t[1] == "Positive":
                    label[t[0]].append(1) #nhan xet tich cuc
                elif t[1] == "Neutral":
                    label[t[0]].append(0) #nhan xet trung binh 
                else:
                    label[t[0]].append(-1) #nhan xet tieu cuc
                    
            else:
                temp.remove(t[0])
                label['OTHERS'].append(1) #phan loai other, ko lien quan
                
        for key in temp:
                label[key].append(-2) #Ko de cap den trong phan nhan xet
    return label
    # for key in label.keys():
    #     print(len(label[key]))
# y = pd.DataFrame(label)
y = spliY(y, label)
y = pd.DataFrame(y)


In [36]:
cleanedData = []
for sentence in X:
    cleanedData.append(sentence)

In [37]:
vectorizer = CountVectorizer(max_features=10000)
BoW = vectorizer.fit_transform(cleanedData)

In [38]:
x_train,x_test,y_train,y_test = train_test_split(BoW,np.asarray(y))

In [39]:
from sklearn.svm import SVC

svm = SVC()
# model.fit(x_train,y_train)
from sklearn.multioutput import MultiOutputClassifier
clf = MultiOutputClassifier(SVC()).fit(x_train, y_train)

In [40]:
# predictions = model.predict(x_test)
predictions = clf.predict(x_test)

In [41]:
def accuracy(y_test, y_pred):
    ac = []
    for j in range(len(y_test)):
        t = 0
        for i in range(len(y_test[j])):
            if y_test[j][i] == y_pred[j][i]:
                t += 1
        ac.append((t/len(y_test[j]))*100)
    return ac
accu_set = accuracy(y_test, predictions)
sum(accu_set) / len(accu_set)

84.2321520287622